In [1]:
from numpy import array
import numpy as np
from numpy import random
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
%matplotlib inline

DATA

In [2]:
data = []
with open('/content/drive/My Drive/cdn1_500m_sigmetrics18.tr' , errors='ignore' ) as fin:
    fin.seek(0)
    d = fin.read(100000000) 

In [3]:
d[99999980:]

'9 881\n4240754 388731'

In [4]:
#Spliting the continous sequence
data = d.split()
data[:10]

['1',
 '778030675',
 '13539',
 '2',
 '3484637698',
 '17635',
 '3',
 '1130267045',
 '152',
 '4']

In [5]:
del d
len(data)

12722261

In [6]:
#We are interested in number of request and request id.
req = []
for i in range(len(data)):
  if (i-1)%3 == 0:
    req.append(int(data[i]))
    #To convert string into real numbers. 

req = np.array(req[:2001000])
req.shape

(2001000,)

In [7]:
np.min(req),np.max(req)

(2301, 4294966263)

In [8]:
id, req1 = np.unique(req, return_inverse= True)
print(len(id))

762195


In [9]:
#To find the range of request id
min = np.min(req1)
max = np.max(req1)
min,max

(0, 762194)

In [10]:
# To reduce total numbers of ID
id = 100
factor = (max - min)/(id-1)
req = np.zeros(len(req1))
for i in range(len(req1)):
  req[i] = int(req1[i]/factor)
req.shape

(2001000,)

In [11]:
np.min(req),np.max(req)

(0.0, 99.0)

In [12]:
window = 1000

In [13]:
count = np.zeros((id),int)
for i in range(window):
    count[int(req[i])] += 1
count.shape

(100,)

In [14]:
prob = np.zeros((len(req)-window,id), dtype = 'f4')

In [15]:
for i in range(len(req)-window):
  count[int(req[i+window])] += 1
  count[int(req[i])] -= 1
  prob[i] = count/window

In [16]:
prob.shape

(2000000, 100)

In [17]:
del req1, data

In [18]:
sample = 100
K = 10
l = 1000000
batch_size = int(l/sample)
encoder_input_data = prob[:l,:].reshape(sample,-1,id) # total samples = 100
decoder_target_data = []
for i in range(sample):
    temp = []
    for j in range(K): # to predict next 10 req for every batch
        temp.append(prob[int((batch_size*i) + j),:])
    decoder_target_data.append(temp)    
decoder_target_data = np.array(decoder_target_data).reshape(sample,K,id)

print(encoder_input_data.shape,decoder_target_data.shape)

(100, 10000, 100) (100, 10, 100)


In [19]:
decoder_input_data = np.zeros(decoder_target_data.shape)
decoder_input_data[1:,:,:] = decoder_target_data[:-1,:,:]
decoder_input_data[0,:,:] = encoder_input_data[-1,-1,:]
decoder_input_data.shape

(100, 10, 100)

MODEL

In [20]:
latent_dim = 100 # LSTM hidden units
dropout = 0.1

In [21]:
encoder_inputs = Input(shape=(None, id)) 
encoder = LSTM(latent_dim, dropout=dropout, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, id)) 
decoder_lstm = LSTM(latent_dim, dropout=dropout, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_outputs = Dense(id, activation='softmax')(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 80400       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 100),  80400       input_2[0][0]                    
                                                                 lstm[0][1]            

In [22]:
model.compile(Adam(), loss='mean_squared_error')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=1,epochs=30)

Epoch 1/30
100/100 [==============================] - 464s 5s/step - loss: 1.2217e-05
Epoch 2/30
100/100 [==============================] - 463s 5s/step - loss: 1.1131e-05
Epoch 3/30
100/100 [==============================] - 462s 5s/step - loss: 1.0395e-05
Epoch 4/30
100/100 [==============================] - 462s 5s/step - loss: 1.0097e-05
Epoch 5/30
100/100 [==============================] - 462s 5s/step - loss: 9.9928e-06
Epoch 6/30
100/100 [==============================] - 462s 5s/step - loss: 9.9539e-06
Epoch 7/30
100/100 [==============================] - 461s 5s/step - loss: 9.9287e-06
Epoch 8/30
100/100 [==============================] - 462s 5s/step - loss: 9.9234e-06
Epoch 9/30
100/100 [==============================] - 462s 5s/step - loss: 9.9101e-06
Epoch 10/30
100/100 [==============================] - 462s 5s/step - loss: 9.8991e-06
Epoch 11/30
100/100 [==============================] - 464s 5s/step - loss: 9.8966e-06
Epoch 12/30
100/100 [==============================]

FUTURE REQUEST PREDICTION

In [23]:
encoder_model = Model(encoder_inputs, encoder_states)
pred_steps = K

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = Dense(id, activation='softmax')(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [24]:
def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, id))
    
    # Populate the first target sequence with end of encoding series pageviews
    target_seq[0, 0, :] = input_seq[0, -1, :]

    # Sampling loop for a batch of sequences - we will fill decoded_seq with predictions
    # (to simplify, here we assume a batch of size 1).

    decoded_seq = np.zeros((1,pred_steps,id))
    for i in range(pred_steps):
        
        output, h, c = decoder_model.predict([target_seq] + states_value)
        
        decoded_seq[0,i,:] = output[0,0,:]

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, id))
        target_seq[0, 0, :] = output[0,0,:]

        # Update states
        states_value = [h, c]

    return decoded_seq

In [25]:
n = int((len(prob) - batch_size)/5000)
n

398

In [26]:
prediction = []
for i in range(5000):
    if i%1000 == 0:
        print(i)
    prediction.append(decode_sequence(prob[i*n:i*n + batch_size,:].reshape(1,batch_size,id)))

0
1000
2000
3000
4000


In [27]:
prediction = np.array(prediction).reshape(5000,K,id)
prediction.shape

(5000, 10, 100)

GENERATION OF FAKE REQUESTS

In [28]:
fake_req = []
for i in range(5000):
    temp1 = []
    for j in range(K):
        temp = []
        for k in range(id):
             temp = (-prediction[i,j]).argsort()[:5]
        for k in range(5):
            temp1.append(temp[k]) 
    fake_req.append(temp1)        
fake_req = np.array(fake_req) 
fake_req.shape

(5000, 50)

In [29]:
len(req) + 5000*50

2251000

In [30]:
gen_req = []
for i in range(batch_size):
    gen_req.append(req[i])
for i in range(50):
    gen_req.append(fake_req[0][i])
for i in range(5000):
    for j in range(n + 51):
        if j <= n:
            gen_req.append(req[batch_size + j + n*i])
        elif(i>0):
            gen_req.append(fake_req[i-1,j-n-1])
gen_req = np.array(gen_req)
gen_req.shape

(2255000,)

In [32]:
req.shape

(2001000,)

In [33]:
np.save('req',req)
np.save('gen_req',gen_req)